### Creating and Persisting an ML Model

In [38]:
import pandas as pd
import numpy as np

Summary of the data

In [39]:
df = pd.read_csv('data/student-mat.csv', sep=';')
df.describe()
df.info

<bound method DataFrame.info of     school sex  age address famsize Pstatus  Medu  Fedu      Mjob      Fjob  \
0       GP   F   18       U     GT3       A     4     4   at_home   teacher   
1       GP   F   17       U     GT3       T     1     1   at_home     other   
2       GP   F   15       U     LE3       T     1     1   at_home     other   
3       GP   F   15       U     GT3       T     4     2    health  services   
4       GP   F   16       U     GT3       T     3     3     other     other   
..     ...  ..  ...     ...     ...     ...   ...   ...       ...       ...   
390     MS   M   20       U     LE3       A     2     2  services  services   
391     MS   M   17       U     LE3       T     3     1  services  services   
392     MS   M   21       R     GT3       T     1     1     other     other   
393     MS   M   18       R     LE3       T     3     2  services     other   
394     MS   M   19       U     LE3       T     1     1     other   at_home   

     ... famrel fre

The goal is to predict the quality of the student. We will build a predictor based on the final grade (G3).
Because we are trying to find quality students. In this model we define a quality student as one who achieves a final grade of 15 or higher. 

Import scikit-learn and build a random forest classifer

In [43]:
import sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score

# Return x (reduced features) and y (qual_student labels)
def parse_data(df):
    df['qual_student'] = np.where(df['G3']>=15, 1, 0)

    # Choose features (drop the G3 score)
    include = ['health','absences','age'] # TODO: change features
    x = df[include]
    y = df['qual_student']
    return x, y

# Print accuracy/F1
def print_scores(y, pred):
        print('accuracy = {},\tF1 = {}'.format( \
                accuracy_score(y, pred), f1_score(y, pred, average='binary')) )

# Build ML model from training data
x, y = parse_data(df)
clf = RandomForestClassifier(n_estimators = 1000)
clf.fit(x, y)

# Generate TRAINING predictions and scores
pred = clf.predict(x)
print_scores(y, pred)

# INITIAL TRAINING: accuracy = 0.8683544303797468, F1 = 0.5357142857142857

accuracy = 0.8683544303797468,	F1 = 0.5185185185185185
accuracy = 0.8683544303797468,	F1 = 0.5185185185185185


It's not very good! We didn't even cross validate. You'll need to do better :)
Let's export this model so we can use it in a microservice (flask api)

In [42]:
import joblib
# TODO: Change these lines
# modify the file path to where you want to save the model
joblib.dump(clf, 'app/handlers/model.pkl')
query_df = pd.DataFrame({ 'age' : pd.Series(1) ,'health' : pd.Series(15) ,'absences' : pd.Series(10)})
pred = clf.predict(query_df)
type(x)

/Users/brandon/miniconda3/envs/se_envir/lib/python3.8/site-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names must be in the same order as they were in fit.

  warnings.warn(message, FutureWarning)


pandas.core.frame.DataFrame